In [1]:
import sys
import os
path = os.path.abspath("../")
sys.path.append(path)
print(path)

/home/jovyan


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, Sampler, random_split, Dataset, Subset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import random

from collections import defaultdict, Counter
from line_profiler import profile
from scipy.linalg import orthogonal_procrustes


import src.helpers.MetricUtilities as metrics
import src.data_processing.ImageProducts as ImageProducts
from functools import partial

from mtree.mtree import MTree
import mtree.mtree as mtree

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

%reload_ext autoreload

%autoreload 2

cpu


In [ ]:
import sys
import os
path = os.path.abspath("../")
sys.path.append(path)

from torch.utils.data import Dataset, Subset
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import random

import src.helpers.MetricUtilities as metrics

from src.helpers.MTreeUtilities import getKNearestNeighbours, getMTree, getMTreeFFT, getMTreeFFTNumba
from src.data_processing.DatasetGetter import get_data, get_data_MStar, get_data_SARDet_100k, get_data_ATRNetSTARAll


import mtree.mtree as mtree

import glob
from PIL import Image

import time

def mtree_ncc_query_sample_size(max_node_size=12, image_size=32, k=7, runs=2, sample_sizes=[1]):
    total_time_ncc = 0
    total_time_mtree = 0
    avg_times_ncc = []
    avg_times_mtree = []

    data = get_data_MStar(image_size)

    print(f"Average runtime of querying mtree and ncc for {k} NN over {runs} runs with image size {image_size} and max node size {max_node_size} and variable sample size")
    for i in range(len(sample_sizes)):
        print(f"NOW TRYING sample size: {sample_sizes[i]}")
        total_time_ncc = 0
        total_time_mtree = 0

        sample_indices = random.sample(range(len(data)), sample_sizes[i])
        sampled_test_data = Subset(data, sample_indices)

        testSample = [item[0] for item in sampled_test_data]
        

        tree = getMTree(testSample, max_node_size)

        # trans = transforms.Compose([transforms.Resize(img_sizes[i])])
        # t_MNIST_data = trans(MNIST_data)

        # for img in MNIST_data:
        #     img = trans(img)

        for _ in range(runs):
            index1 = np.random.randint(len(data))
            #input1=input_dataset[index1][0].squeeze().to('cpu')
            unseen_image = data[index1][0]

            start_time = time.time()
            arr = []
            for j in range(len(testSample)):
                result = ImageProducts.ncc_scaled(testSample[j], unseen_image)
                arr.append(result)
            
            unseen_img_arr = np.array(arr)
            #print(unseen_img_arr)
            imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
            end_time = time.time()

            total_time_ncc += end_time - start_time

            start_time = time.time()
            imgs = getKNearestNeighbours(tree, unseen_image, k+1)
            end_time = time.time()
            total_time_mtree += end_time - start_time

        avg_ncc = total_time_ncc / runs
        avg_mtree = total_time_mtree / runs
        avg_times_ncc.append(avg_ncc)
        avg_times_mtree.append(avg_mtree)

        # with open("test_mtree_query_sample_sizes_avg_times_ncc_.txt", "w") as file:
        #     file.write(f"avg times: {str(avg_times_ncc)}")
        
        # with open("test_mtree_query_sample_sizes_avg_times_mtree_.txt", "w") as file:
        #     file.write(f"avg times: {str(avg_times_mtree)}")

        print(f"Average runtime of ncc search: {avg_ncc:.6f} seconds")
        print(f"Average runtime of mtree search: {avg_mtree:.6f} seconds")
    # plot_data_mtree_ncc(x_axis="Sample size", title=f"Average runtime of finding {k} neighbours with image size {image_size} against sample sizes", filename="test_mtree_query_sample_sizes.png", 
    #                     varied_arr=sample_sizes, data1=avg_times_ncc, data2=avg_times_mtree, max_node_size=max_node_size)


def mtree_init_sample_size(max_node_size=12, image_size=32, k=7, runs=2, sample_sizes=[1]):
    total_time_mtree_init = 0
    avg_times_mtree_init = []

    data = get_data_MStar(image_size)

    print(f"Average runtime of initialising mtree over {runs} runs with max node size {max_node_size} and image size {image_size} and variable sample sizes")

    for i in range(len(sample_sizes)):
        print(f"NOW TRYING sample size: {sample_sizes[i]}")
        total_time_mtree_init = 0

        sample_indices = random.sample(range(len(data)), sample_sizes[i])
        sampled_test_data = Subset(data, sample_indices)

        testSample = [item[0] for item in sampled_test_data]
        #sampled_test_data[:,0]

        for _ in range(runs):
            start_time = time.time()
            tree = getMTree(testSample, max_node_size)
            end_time = time.time()
            total_time_mtree_init += end_time - start_time
            print(f"Finished one run of mtree init...")

        
        avg_mtree_init = total_time_mtree_init / runs
        avg_times_mtree_init.append(avg_mtree_init)

        
        # with open("test_mtree_init_sample_sizes_avg_times_mtree_init.txt", "w") as file:
        #     file.write(f"avg times: {str(avg_times_mtree_init)}")

        print(f"Average runtime of mtree init: {avg_mtree_init:.6f} seconds")
    
    # plot_data_mtree_init(x_axis="Sample size", title=f"Average runtime of initialising mtree with image size {image_size}, max node size {max_node_size} against sample sizes", filename="test_mtree_init_sample_sizes.png", 
    #                     varied_arr=sample_sizes, data=avg_times_mtree_init, max_node_size=max_node_size)

In [ ]:
# 9466 samples
# note MStar is 368 by 368...

mtree_ncc_query_sample_size(max_node_size=15, image_size=368, k=7, runs=10, sample_sizes=[10, 100, 1000, 2000])

In [ ]:
import json

with open("../data/SARDet_100k/SARDet_100K/Annotations/val.json") as val_annotations:
    mappings = json.load(val_annotations)

print(mappings["images"][0])
print(len(mappings["images"]))

In [4]:
'''
@inproceedings{li2024sardet100k,
	title={SARDet-100K: Towards Open-Source Benchmark and ToolKit for Large-Scale SAR Object Detection}, 
	author={Yuxuan Li and Xiang Li and Weijie Li and Qibin Hou and Li Liu and Ming-Ming Cheng and Jian Yang},
	year={2024},
	booktitle={The Thirty-eighth Annual Conference on Neural Information Processing Systems (NeurIPS)},
}

@article{zhang2025rsar,
  title={RSAR: Restricted State Angle Resolver and Rotated SAR Benchmark},
  author={Zhang, Xin and Yang, Xue and Li, Yuxuan and Yang, Jian and Cheng, Ming-Ming and Li, Xiang},
  journal={Proceedings of the IEEE/CVF conference on computer vision and pattern recognition},
  year={2025}
}

@article{dai2024denodet,
	title={DenoDet: Attention as Deformable Multi-Subspace Feature Denoising for Target Detection in SAR Images},
	author={Dai, Yimian and Zou, Minrui and Li, Yuxuan and Li, Xiang and Ni, Kang and Yang, Jian},
	journal={arXiv preprint arXiv:2406.02833},
	year={2024}
}
'''
import json
# TODO look at train/val/test.json format and extract out the category id and the id to class mapping at the v end.


class CustomDatasetSARDet_100k(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.data = []
        
        with open("../data/SARDet_100k/SARDet_100K/Annotations/test.json") as annotations:
            test_mapping = json.load(annotations)
        with open("../data/SARDet_100k/SARDet_100K/Annotations/train.json") as annotations:
            train_mapping = json.load(annotations)
        with open("../data/SARDet_100k/SARDet_100K/Annotations/val.json") as annotations:
            val_mapping = json.load(annotations)
        
        self.imgs_path = "../data/SARDet_100k/SARDet_100K/JPEGImages/"

        test_path = self.imgs_path + "test/"
        train_path = self.imgs_path + "train/"
        val_path = self.imgs_path + "val/"

        for i in range(len(test_mapping["images"])):
            img_path = test_path + test_mapping["images"][i]["file_name"]
            cat_id = test_mapping["annotations"][i]["category_id"]
            cat_name = test_mapping["categories"][int(cat_id)]["name"]
            self.data.append([img_path, cat_name])

        for i in range(len(train_mapping["images"])):
            img_path = train_path + train_mapping["images"][i]["file_name"]
            cat_id = train_mapping["annotations"][i]["category_id"]
            cat_name = train_mapping["categories"][int(cat_id)]["name"]
            self.data.append([img_path, cat_name])
        
        for i in range(len(val_mapping["images"])):
            img_path = val_path + val_mapping["images"][i]["file_name"]
            cat_id = val_mapping["annotations"][i]["category_id"]
            cat_name = val_mapping["categories"][int(cat_id)]["name"]
            self.data.append([img_path, cat_name])

        # file_list = glob.glob(self.imgs_path + "*")
        # self.data = []

        # # with open("../data/SARDet_100k/SARDet_100K/mapping.json") as annotations:
        # #     mappings = json.load(annotations)

        # for dir_path in file_list:
        #     for img_path in glob.glob(dir_path + "/*.png"):
        #         # self.data.append([img_path, mappings[img_path.split("/")[-1]]])
        #         self.data.append([img_path, "yippee"])
        #     for img_path in glob.glob(dir_path + "/*.jpg"):
        #         # self.data.append([img_path, mappings[img_path.split("/")[-1]]])
        #         self.data.append([img_path, "yippee"])
        

    # Defining the length of the dataset
    def __len__(self):
        return len(self.data)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        data_path = self.data[index]
        image = Image.open(data_path[0])
        image = transforms.functional.to_grayscale(image)
        # class_id = self.class_map[data_path[1]]
        class_id = data_path[1]
        # Applying the transform
        if self.transform:
            image = self.transform(image)
        
        return image.squeeze().to('cpu').numpy(), class_id

def get_data_SARDet_100k(size):
    transform = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])
    return CustomDatasetSARDet_100k(transform)


In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse('/home/jovyan/data/ATRNet-STAR/EOC_azimuth/test/Buick_Excelle_GT/KU_HH_15_80_325790.xml')
root = tree.getroot()

print("Root tag:", root.tag)


img_path = root.find('filename').text
broad_class = root.find('object').find('class').text
subclass = root.find('object').find('subclass').text
subclass_type = "_".join(root.find('object').find('type').text.split("_")[1:])

print(img_path)
print(broad_class)
print(subclass)
print(subclass_type)

In [ ]:
'''
misc{liu2025atrnet,
    title={{ATRNet-STAR}: A Large Dataset and Benchmark Towards Remote Sensing Object Recognition in the Wild}, 
    author={Yongxiang Liu and Weijie Li and Li Liu and Jie Zhou and Bowen Peng and Yafei Song and Xuying Xiong and Wei Yang and Tianpeng Liu and Zhen Liu and Xiang Li},
    year={2025},
    eprint={2501.13354},
    archivePrefix={arXiv},
    primaryClass={cs.CV},
    url={https://arxiv.org/abs/2501.13354},
}

@ARTICLE{li2025saratr,
  author={Li, Weijie and Yang, Wei and Hou, Yuenan and Liu, Li and Liu, Yongxiang and Li, Xiang},
  journal={IEEE Transactions on Image Processing}, 
  title={SARATR-X: Toward Building a Foundation Model for SAR Target Recognition}, 
  year={2025},
  volume={34},
  number={},
  pages={869-884},
  doi={10.1109/TIP.2025.3531988}}

@ARTICLE{li2024predicting,
  title = {Predicting gradient is better: Exploring self-supervised learning for SAR ATR with a joint-embedding predictive architecture},
  journal = {ISPRS Journal of Photogrammetry and Remote Sensing},
  volume = {218},
  pages = {326-338},
  year = {2024},
  issn = {0924-2716},
  doi = {https://doi.org/10.1016/j.isprsjprs.2024.09.013},
  url = {https://www.sciencedirect.com/science/article/pii/S0924271624003514},
  author = {Li, Weijie and Yang, Wei and Liu, Tianpeng and Hou, Yuenan and Li, Yuxuan and Liu, Zhen and Liu, Yongxiang and Liu, Li}}

Downloads last month
167
System
'''
import xml.etree.ElementTree as ET



class CustomDatasetATRNetSTAR(Dataset):
    def __init__(self, transform=None, p_left=0.8, p_right=0.2):
        # Wall_poer real name is j poer. J for easier classname extraction... and cheetah and voleex
        self.type_names = ["Excelle_GT", "GL8", "CS75_Plus", "Starlight_4500", "Cheetah_CFA6473C", "8228-5", "Arrizo 5", "qq3", "Blazer_1998", "HOWO", "Duolika", "EQ6608LTV", "Forthing_Lingzhi",
        "Tianjin_DFH2200B", "Tianjin_KR230", "J6P", "Jiabao_T51", "BJ1045V9JB5-54", "Wall_poer", "Wall_Voleex_C50", "EV160B", "CA7180A3E", "h5", "N1", "HLF25_II", "Junling", "Patriot", 
        "SY5033XJH", "MKC", "Proud_2009", "V80", "Outlander_2003", "ZL40F", "DeLong_M3000", "DeLong_X3000", "Aochi_1800", "Aochi_Hongrui", "Rongguang_V", "YZK6590XCA", "ZK6120HY1"]
        self.categories = {}
        for i in range(len(self.type_names)):
            self.categories[self.type_names[i]] = []
        self.transform = transform
        self.data = []
        self.test = []
        # Check if I can test the azimuths.... if not j try smt else idk
        self.imgs_path = "/home/jovyan/data/ATRNet-STAR/EOC_azimuth/"
        #file_list = [glob.glob(self.imgs_path + "*")]
        file_list = [self.imgs_path + "test_300/"]
        print(file_list)
        for dir_path in file_list:
            #print(dir_path)
            for cat in glob.glob(dir_path + "/*"):
                #print(cat)
                for xml_file_path in glob.glob(cat + "/*.xml"):
                    #print(xml_file_path)
                    tree = ET.parse(xml_file_path)
                    root = tree.getroot()
                    img_path = root.find('filename').text
                    broad_class = root.find('object').find('class').text
                    subclass = root.find('object').find('subclass').text
                    subclass_type = "_".join(root.find('object').find('type').text.split("_")[1:])
                    chip_data = [img_path, broad_class, subclass, subclass_type] 
                    # adding chipdata to the dictionary entry w that class name
                    self.categories[subclass_type] = self.categories[subclass_type] + chip_data
                print(self.categories)
                    # self.data.append(chip_data)

        print(self.categories)
        self.data, self.test = self.split_data(p_left, p_right)


    # Defining the length of the dataset
    def __len__(self):
        return len(self.data) + len(self.test)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        data_path = self.data[index]
        image = Image.open(data_path[0])
        image = transforms.functional.to_grayscale(image)
        # class_id = self.class_map[data_path[1]]
        broad_class = data_path[1]
        subclass = data_path[2]
        subclass_type = data_path[3]
        # Applying the transform
        if self.transform:
            image = self.transform(image)
        
        return image.squeeze().to('cpu').numpy(), broad_class, subclass, subclass_type
    
    def split_data(self, p_left, p_right):
        data = [["head", "head", "head", "head"]]
        test = [["head", "head", "head", "head"]]
        for key in self.categories:
            entry = np.array(self.categories[key])
            print(entry)
            print(type(entry))
            left_len = math.floor(p_left * len(entry))
            right_len = math.ceil(p_right * len(entry))
            sample_indices = np.array(random.sample(range(len(entry)), left_len))
            print(sample_indices)
            # sampled_test_data = Subset(data, sample_indices)

            mask = np.ones(len(entry), np.bool)
            mask[sample_indices] = 0
            # These are all wrong omg ded
            left_partition = entry[sample_indices]
            right_partition = entry[mask]
            print(left_partition)
            print(right_partition)
            data += left_partition
            test += right_partition
            # other_indices = np.arange(len(entry))[mask]
            # left_partition = Subset(entry, sample_indices)
            # right_partition = Subset(entry, )
            # other_data = data[mask]
        
        return data, test




def get_data_ATRNetSTAR(size, p_left=0.8, p_right=0.2):
    transform = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])
    return CustomDatasetATRNetSTAR(transform, p_left, p_right)


In [ ]:
data = get_data_ATRNetSTAR(100, 0.8, 0.2)
print(len(data))


In [ ]:
data = get_data_SARDet_100k(300)
print(len(data))
sample_indices = random.sample(range(len(data)), 100)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
classes = [item[1] for item in sampled_test_data]


In [ ]:
index1 = np.random.randint(len(testSample))
#index2 = np.random.randint(len(testSample))
#input2=input_dataset[index2][0].squeeze().to('cpu')
print(classes[index1])
plt.imshow(testSample[index1], cmap='grey')
plt.show()


In [ ]:

k=7
data = get_data_MStar(368)
print(len(data))
sample_indices = random.sample(range(len(data)), 100)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
classes = [item[1] for item in sampled_test_data]

# class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

tree = getMTree(testSample, 12)

index1 = random.choice(sample_indices)
#input1=input_dataset[index1][0].squeeze().to('cpu')
unseen_image = data[index1][0]
unseen_image_class = data[index1][1]

arr = []
for j in range(len(testSample)):
    result = ImageProducts.ncc_scaled(testSample[j], unseen_image)
    arr.append(result)

unseen_img_arr = np.array(arr)
#print(unseen_img_arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]

num_same_class = 0

print(f"Unseen img class: {unseen_image_class}")
for i in imgProd_max_index:
    img_class = classes[i]
    print(f"Index {i}, Class {img_class}")
    if (img_class == unseen_image_class):
        num_same_class += 1

print(num_same_class)


imgs = getKNearestNeighbours(tree, unseen_image, k+1)


def imgs_to_indices(img_arr, testSample):
    ind_arr = []
    for img in img_arr:
        for i in range(len(testSample)):
            if (metrics.distance(img, testSample[i]) < 0.00001):
                ind_arr.append(i)
                break
    
    return ind_arr

ind_arr = imgs_to_indices(imgs, testSample)
print(ind_arr)
print(imgProd_max_index)


In [ ]:
import numpy as np
import cv2

def ncc(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc

    Applies NCC of the template image over the main image and returns the max value obtained.
    When the template image kernel exceeds the bounds, wraps to the other side of the main image
    """
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)

    corr = cv2.matchTemplate(mainImg, tempImg, cv2.TM_CCORR_NORMED)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corr)

    return max_val

def ncc_scaled(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc, with scaled bounds of [-1,1]
    """
    return ncc(mainImg, tempImg) * 2 - 1


class CustomDatasetMStar(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        self.imgs_path = "../data/mstar/Padded_imgs/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.JPG"):
                self.data.append([img_path, class_name])
        #print(self.data)
        self.class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

    # Defining the length of the dataset
    def __len__(self):
        return len(self.data)

    # Defining the method to get an item from the dataset
    def __getitem__(self, index):
        data_path = self.data[index]
        image = Image.open(data_path[0])
        image = transforms.functional.to_grayscale(image)
        class_id = self.class_map[data_path[1]]
        # Applying the transform
        if self.transform:
            image = self.transform(image)
        
        return image.squeeze().to('cpu').numpy(), class_id

k=7
data = get_data_MStar(16)
print(len(data))
sample_indices = random.sample(range(len(data)), 100)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
classes = [item[1] for item in sampled_test_data]

# class_map = {"2S1" : 0, "BRDM_2": 1, "BTR_60": 2, "D7": 3, "SLICY": 4, "T62": 5, "ZIL131": 6, "ZSU_23_4": 7}

tree = getMTreeFFTNumba(testSample, 12)
tree_mst = getMTreeFFTNumba(testSample, 12, promote=mtree.MST_promotion, partition=mtree.MST_partition)

index1 = random.choice(sample_indices)
#input1=input_dataset[index1][0].squeeze().to('cpu')
unseen_image = data[index1][0]
unseen_image_class = data[index1][1]

arr = []
for j in range(len(testSample)):
    result = ncc_scaled(testSample[j], unseen_image)
    arr.append(result)

unseen_img_arr = np.array(arr)
#print(unseen_img_arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]

num_same_class = 0

print(f"Unseen img class: {unseen_image_class}")
for i in imgProd_max_index:
    img_class = classes[i]
    print(f"Index {i}, Class {img_class}")
    if (img_class == unseen_image_class):
        num_same_class += 1

print(num_same_class)


imgs = getKNearestNeighbours(tree, unseen_image, k+1)
imgs_mst = getKNearestNeighbours(tree_mst, unseen_image, k+1)

def imgs_to_indices(img_arr, testSample):
    ind_arr = []
    for img in img_arr:
        for i in range(len(testSample)):
            if (metrics.dist_fft_numba(img, testSample[i]) < 0.0000001):
                ind_arr.append(i)
                break
    
    return ind_arr

ind_arr = imgs_to_indices(imgs, testSample)
ind_arr_mst = imgs_to_indices(imgs_mst, testSample)
print(ind_arr_mst)
print(ind_arr)
print(imgProd_max_index)


In [ ]:
import scipy.fft
import math

def ncc(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc

    Applies NCC of the template image over the main image and returns the max value obtained.
    When the template image kernel exceeds the bounds, wraps to the other side of the main image
    """
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)

    corr = cv2.matchTemplate(mainImg, tempImg, cv2.TM_CCORR_NORMED)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corr)

    return max_val

def ncc_unnormed(mainImg, tempImg) -> float:
    """
    :param mainImg: Main image to be scanned
    :param tempImg: Template image to be scanned over the main
    :return: Max value of the ncc

    Applies NCC of the template image over the main image and returns the max value obtained.
    When the template image kernel exceeds the bounds, wraps to the other side of the main image
    """
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)

    corr = cv2.matchTemplate(mainImg, tempImg, cv2.TM_CCORR)

    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corr)

    return max_val

dataset = get_data_MStar(300)

sample_indices = random.sample(range(len(data)), 100)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
index1 = np.random.randint(len(data))
index2 = np.random.randint(len(testSample))
unseen_image = data[index1][0]


test_img = testSample[index2]
test_img2 = unseen_image
m = test_img.shape[0]

ncc_score_un = ncc_unnormed(test_img, test_img2)
ncc_score = ncc(test_img, test_img2)
print(ncc_score)


def ncc_fft(mainImg, tempImg):
    A = scipy.fft.fft2(mainImg)
    B = scipy.fft.fft2(tempImg)

    Z = scipy.fft.ifft2(np.conj(A) * B).real

    auto_A = scipy.fft.ifft2(np.conj(A) * A).real
    auto_B = scipy.fft.ifft2(np.conj(B) * B).real
    auto_A = np.maximum(auto_A, 0.0001)
    auto_B = np.maximum(auto_B, 0.0001)
    auto_A_sqrt = np.sqrt(auto_A)
    auto_B_sqrt = np.sqrt(auto_B)
    denom = np.multiply(auto_A_sqrt, auto_B_sqrt).max() # (THIS WORKS.) (Basically uhm so the normalization amt should be the same throughout? I'm not v sure why this works...)

    return np.divide(Z, denom).max()

#print(ncc_fft(test_img, test_img2))

def ncc_naive(mainImg, tempImg):
    if np.count_nonzero(mainImg) == 0:
        if np.count_nonzero(tempImg) == 0:
            return 1
        return 0

    mainImg = np.pad(mainImg, max(len(mainImg), len(mainImg[0])),
                     'wrap')  # Padding the main image with wrapped values to simulate wrapping

    mainImg = np.asarray(mainImg, np.single)  # Setting data types of array
    tempImg = np.asarray(tempImg, np.single)
    m = tempImg.shape[0]
    ncc_arr = np.ones((2*m, 2*m))
    for i in range(2 * m):
        for j in range(2 * m):
            sum = 0
            sum_norm_main = 0
            sum_norm_temp = 0
            for x in range(m):
                for y in range(m):
                    sum += mainImg[x + i][y + j] * tempImg[x][y]
                    sum_norm_main += (mainImg[x+i][y+j])**2
                    sum_norm_temp += (tempImg[x][y])**2
            
            ncc_arr[i][j] = (sum) / (math.sqrt(sum_norm_main * sum_norm_temp))
    
    return ncc_arr.max()

def ncc_rfft(mainImg, tempImg):
    A = scipy.fft.rfft2(mainImg)
    B = scipy.fft.rfft2(tempImg)

    Z = scipy.fft.irfft2(np.conj(A) * B).real

    auto_A = scipy.fft.irfft2(np.conj(A) * A).real.max()
    auto_B = scipy.fft.irfft2(np.conj(B) * B).real.max()
    auto_A = np.maximum(auto_A, 0.0001)
    auto_B = np.maximum(auto_B, 0.0001)
    auto_A_sqrt = np.sqrt(auto_A)
    auto_B_sqrt = np.sqrt(auto_B)
    denom = np.multiply(auto_A_sqrt, auto_B_sqrt).max() # (THIS WORKS.) (Basically uhm so the normalization amt should be the same throughout? I'm not v sure why this works...)

    return np.divide(Z, denom).max()

print(ncc_rfft(test_img, test_img2))
print(ncc_fft(test_img, test_img2))

# for i in range(100):
#     sample_indices = random.sample(range(len(data)), 100)
#     sampled_test_data = Subset(data, sample_indices)

#     testSample = [item[0] for item in sampled_test_data]
#     index1 = np.random.randint(len(data))
#     index2 = np.random.randint(len(testSample))
#     unseen_image = data[index1][0]


#     test_img = testSample[index2]
#     test_img2 = unseen_image
#     m = test_img.shape[0]

#     ncc_score = ncc(test_img, test_img2)
#     ncc_fft_score = ncc_fft(test_img, test_img2)
#     ncc_rfft_score = ncc_rfft(test_img, test_img2)
#     if (abs(ncc_score - ncc_fft_score) > 0.00001 or abs(ncc_score - ncc_rfft_score) > 0.00001):
#         print(ncc_score)
#         print(ncc_fft_score)
#         print(ncc_rfft_score)

# print(ncc_naive(test_img, test_img2))

## TEST NUMBA

In [ ]:
import numba as nb

@nb.njit
def ncc_fft_numba(mainImg, tempImg):
    A = scipy.fft.fft2(mainImg)
    B = scipy.fft.fft2(tempImg)

    Z = scipy.fft.ifft2(np.conj(A) * B).real

    auto_A = scipy.fft.ifft2(np.conj(A) * A).real
    auto_B = scipy.fft.ifft2(np.conj(B) * B).real
    auto_A = np.maximum(auto_A, 0.0001)
    auto_B = np.maximum(auto_B, 0.0001)
    auto_A_sqrt = np.sqrt(auto_A)
    auto_B_sqrt = np.sqrt(auto_B)
    denom = np.multiply(auto_A_sqrt, auto_B_sqrt).max()

    return np.divide(Z, denom).max()

@nb.njit
def ncc_rfft_numba(mainImg, tempImg):
    A = scipy.fft.rfft2(mainImg)
    B = scipy.fft.rfft2(tempImg)

    Z = scipy.fft.irfft2(np.conj(A) * B).real

    auto_A = scipy.fft.irfft2((np.conj(A) * A).real)
    auto_B = scipy.fft.irfft2((np.conj(B) * B).real)
    auto_A = np.maximum(auto_A, 0.0001)
    auto_B = np.maximum(auto_B, 0.0001)
    auto_A_sqrt = np.sqrt(auto_A)
    auto_B_sqrt = np.sqrt(auto_B)
    denom = np.multiply(auto_A_sqrt, auto_B_sqrt).max()

    return np.divide(Z, denom).max()

In [ ]:
# initialising

ncc_score = ncc_fft_numba(test_img, test_img2)
ncc_score = ncc_rfft_numba(test_img, test_img2)

In [ ]:
import time

runs = 1000
ttime = 0

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc_fft_numba(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"rocket-fft ncc fft runtime: {ttime / runs}")

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc_fft(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"ncc fft runtime: {ttime / runs}")

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"cv2 ncc runtime: {ttime / runs}")

for i in range(runs):
    start_time = time.perf_counter()
    ncc_score = ncc_rfft_numba(test_img, test_img2)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"ncc rfft rocket runtime: {ttime / runs}")


In [ ]:
from numba import cuda, vectorize, float64


# @cuda.jit()
# def linear_ncc_search(testSample, unseen_image, k, arr):
#     i = cuda.grid(1)
#     if (i < len(testSample)):
#         arr[i] = ncc_fft_numba(testSample[i], unseen_image)

@vectorize([float64(float64, float64)], cache=True)
def f(x, y):
    return ncc_fft_numba(x, y)

In [ ]:

data = get_data_MStar(32)

sample_indices = random.sample(range(len(data)), 50)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
print(len(testSample))
index1 = np.random.randint(len(data))
#input1=input_dataset[index1][0].squeeze().to('cpu')
unseen_image = data[index1][0]
unseen_images = np.array([unseen_image for i in range(len(testSample))])


k=7

# x_host = np.ones(shape=(len(testSample)))
# x_device = cuda.to_device(x_host)
# threadsperblock = 256
# blockspergrid = (x_device.size + (threadsperblock - 1)) // threadsperblock

In [ ]:

start_time = time.perf_counter()
# arr = [-1 for i in range(len(testSample))]
# linear_ncc_search[blockspergrid, threadsperblock](testSample, unseen_image, k, arr)
unseen_img_arr = f(testSample, unseen_images)
#unseen_img_arr = np.array(x_device)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
end_time = time.perf_counter()

print(f"time: {end_time - start_time}")

In [ ]:
from numba import prange

@nb.njit(parallel=True)
def linear_ncc_psearch(testSample, unseen_image, arr):
    for i in prange(len(testSample)):
        arr[i] = ncc_fft_numba(testSample[i], unseen_image)

    return arr

@nb.njit()
def linear_ncc_search(testSample, unseen_image, arr):
    for i in range(len(testSample)):
        arr[i] = ncc_fft_numba(testSample[i], unseen_image)

    return arr



In [ ]:
from numba import cuda

# Measures free memory bef and aft allocation to device.
print(cuda.select_device(0))
print(cuda.current_context().get_memory_info()[0])
arr = np.zeros(10**9, dtype = 'float32')
d_ary = cuda.to_device(arr)
print(cuda.current_context().get_memory_info()[0])
print(d_ary)



In [ ]:
runs = 1
ttime = 0
image_size = 100
sample_size = 9000

# data = get_data_MStar(image_size)

# sample_indices = random.sample(range(len(data)), sample_size)
# sampled_test_data = Subset(data, sample_indices)

# testSample = [item[0] for item in sampled_test_data]
# index1 = np.random.randint(len(data))
# #input1=input_dataset[index1][0].squeeze().to('cpu')
# unseen_image = data[index1][0]

data = get_data_MStar(image_size)
sample_indices = random.sample(range(len(data)), sample_size)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
index1 = np.random.randint(len(data))
unseen_image = data[index1]

In [ ]:
testSample = [item[0] for item in sampled_test_data]
testSample = np.array(testSample)
print(testSample.shape)

In [ ]:
ncc_fft(testSample[0], testSample[-1])

In [ ]:
k = 7
runs = 100
ttime = 0
image_size = 16
sample_size = 20000

# data = get_data_SARDet_100k(image_size)

# sample_indices = random.sample(range(len(data)), sample_size)
# sampled_test_data = Subset(data, sample_indices)

# testSample = [item[0] for item in sampled_test_data]
# classes = [item[1] for item in sampled_test_data]
# testSampleIndexed = [(item, i) for item in testSample]
# index1 = np.random.randint(len(data))
# # #input1=input_dataset[index1][0].squeeze().to('cpu')
# unseen_image = data[index1][0]
# unseen_image_class = data[index1][1]

print(f"Unseen image class: {unseen_image_class}")
# print()

arr = np.ones(len(testSample))
unseen_img_arr = linear_ncc_psearch(testSample, unseen_image, arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]

other = linear_ncc_search(testSample, unseen_image, arr)
other_index = np.argpartition(other, -(k+1))[-(k+1):]
# print()
for i in range(len(imgProd_max_index)):
    print(classes[imgProd_max_index[i]])

for i in range(len(other_index)):
    print(classes[other_index[i]])

# data = get_data(image_size)
# sample_indices = random.sample(range(len(data)), sample_size)
# sampled_test_data = Subset(data, sample_indices)

# testSample = [item for item in sampled_test_data]
# index1 = np.random.randint(len(data))
# unseen_image = data[index1]

# for i in range(runs):
#     start_time = time.perf_counter()
#     arr = np.ones(len(testSample))
#     unseen_img_arr = linear_ncc_psearch(testSample, unseen_image, arr)
#     imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
#     end_time = time.perf_counter()
#     ttime += end_time - start_time

# print(f"time taken for parallelised linear ncc search with rocket fft over {runs} runs for image size {image_size} and sample size {sample_size} : {ttime / runs}")





In [ ]:
runs = 1
ttime = 0
image_size = 16
sample_size = 100000
# data = get_data_MStar(image_size)

# sample_indices = random.sample(range(len(data)), sample_size)
# sampled_test_data = Subset(data, sample_indices)

# testSample = [item[0] for item in sampled_test_data]
# index1 = np.random.randint(len(data))
# unseen_image = data[index1][0]

# data = get_data(image_size)
# sample_indices = random.sample(range(len(data)), sample_size)
# sampled_test_data = Subset(data, sample_indices)

# testSample = [item for item in sampled_test_data]
# index1 = np.random.randint(len(data))
# unseen_image = data[index1]

for i in range(runs):
    start_time = time.perf_counter()
    arr = np.ones(len(testSample))
    unseen_img_arr = linear_ncc_search(testSample, unseen_image, arr)
    imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"time taken for linear ncc search with rocket fft over {runs} runs for image size {image_size} and sample size {sample_size} : {ttime / runs}")

In [ ]:
runs = 10
ttime = 0
image_size = 16
sample_size = 100
data = get_data_SARDet_100k(image_size)

sample_indices = random.sample(range(len(data)), sample_size)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
index1 = np.random.randint(len(data))
#input1=input_dataset[index1][0].squeeze().to('cpu')
unseen_image = data[index1][0]

start_time = time.perf_counter()
tree = getMTreeFFT(testSample, 12)
end_time = time.perf_counter()

print(f"time taken for mtree init with rocket fft over {1} runs for image size {image_size} and sample size {sample_size} : {end_time - start_time}")

for i in range(runs):
    index1 = np.random.randint(len(data))
    #input1=input_dataset[index1][0].squeeze().to('cpu')
    unseen_image = data[index1][0]
    start_time = time.perf_counter()
    arr = np.ones(len(testSample))
    img_arr = getKNearestNeighbours(tree, unseen_image, k)
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"time taken for mtree with rocket fft over {runs} runs for image size {image_size} and sample size {sample_size} : {ttime / runs}")

In [ ]:
runs = 1
ttime = 0
image_size = 16
sample_size = 100

data = get_data_MStar(image_size)

sample_indices = random.sample(range(len(data)), sample_size)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
index1 = np.random.randint(len(data))
unseen_image = data[index1][0]

# data = get_data(image_size)
# sample_indices = random.sample(range(len(data)), sample_size)
# sampled_test_data = Subset(data, sample_indices)

# testSample = [item for item in sampled_test_data]
# index1 = np.random.randint(len(data))
# unseen_image = data[index1]

for i in range(runs):
    start_time = time.perf_counter()
    arr = np.ones(len(testSample))
    unseen_img_arr = linear_ncc_search(testSample, unseen_image, arr)
    imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
    end_time = time.perf_counter()
    ttime += end_time - start_time

print(f"time taken for linear ncc search with rocket fft over {runs} runs for image size {image_size} and sample size {sample_size} : {ttime / runs}")

## Accuracy test sanity check

In [ ]:
k=3
runs = 100
ttime = 0
image_size = 16
sample_size = 100000

def imgs_to_indices(img_arr, testSample):
    added = False
    ind_arr = []
    for img in img_arr:
        added = False
        for i in range(len(testSample)):
            if (metrics.distance(img, testSample[i]) < 1e-7):
                ind_arr.append(i)
                added = True
                break
        if (not added):
            print("NOT ADDED")
            print(f"ncc: {ncc(img,img)}")
            print(f"img prod ncc: {ImageProducts.ncc(img, img)}")
            print(f"ncc fft: {ncc_fft_numba(img, img)}")
            print(metrics.distance(img, img))
            for j in range(len(testSample)):
                print(metrics.distance(img, testSample[j]))
                print(f"ncc against test samples: {ncc(img,testSample[j])}")

    return ind_arr

data = get_data_SARDet_100k(image_size)


for i in range(runs):
    sample_indices = random.sample(range(len(data)), sample_size)
    sampled_test_data = Subset(data, sample_indices)

    testSample = [item[0] for item in sampled_test_data]
    testSampleIndexed = []
    for i in range(len(testSample)):
        testSampleIndexed.append((testSample[i], i))
    index1 = np.random.choice(sample_indices)
    unseen_image = data[index1][0]
    unseen_image_indexed = [unseen_image, index1]
    
    arr = np.ones(len(testSample))
    unseen_img_parr = linear_ncc_psearch(testSample, unseen_image, arr)
    imgProd_max_pindex = np.argpartition(unseen_img_parr, -(k+1))[-(k+1):]

    # arr = []
    # for j in range(len(testSample)):
    #     result = ncc(testSample[j], unseen_image)
    #     arr.append(result)

    # unseen_img_arr_normal = np.array(arr)
    # #print(unseen_img_arr)
    # imgProd_max_index_normal = np.argpartition(unseen_img_arr_normal, -(k+1))[-(k+1):]

    # arr = []
    # for j in range(len(testSample)):
    #     result = ncc_fft(testSample[j], unseen_image)
    #     arr.append(result)

    unseen_img_arr_fft = np.array(arr)
    #print(unseen_img_arr)
    imgProd_max_index_fft = np.argpartition(unseen_img_arr_fft, -(k+1))[-(k+1):]

    # tree = getMTreeFFTNumba(testSample, 12)
    # img_arr = getKNearestNeighbours(tree, unseen_image, k)

    # ind_arr = imgs_to_indices(img_arr, testSample)
    # print(len(img_arr))

    normal_tree = getMTree(testSampleIndexed, 3, promote=mtree.MST_promotion, partition=mtree.MST_partition, d=metrics.dist_fft_numba_indexed)
    # print(len(normal_tree))
    img_arr_2 = getKNearestNeighbours(normal_tree, unseen_image_indexed, k=3)
    print(len(img_arr_2))
    img_arr_2 = [item[1] for item in img_arr_2]

    #print(img_arr_2)
    # print(len(img_arr_2))    
    # ind_arr_2 = imgs_to_indices(img_arr_2, testSample)

    # mtree_fft = getMTreeFFT(testSample, 12)
    # img_arr_3 = getKNearestNeighbours(tree, unseen_image, k)
    # ind_arr_3 = imgs_to_indices(img_arr_3, testSample)

    # tree_mst = getMTreeFFTNumba(testSample, 12, promote=mtree.MST_promotion, partition=mtree.MST_partition)
    # img_arr = getKNearestNeighbours(tree_mst, unseen_image, k)
    # ind_arr_3 = imgs_to_indices(img_arr, testSample)

    # tree_mst_normal = getMTree(testSample, 12, promote=mtree.MST_promotion, partition=mtree.MST_partition)
    # img_arr = getKNearestNeighbours(tree_mst_normal, unseen_image, k)
    # ind_arr_4 = imgs_to_indices(img_arr, testSample)


    # imgProd_max_index = set(list(imgProd_max_index[1:].astype(int)))
    # imgProd_max_index_normal = set(list(imgProd_max_index_normal[1:].astype(int)))
    # imgProd_max_index_fft = set(list(imgProd_max_index_fft[1:].astype(int)))
    #imgProd_max_pindex = set(list(imgProd_max_pindex[1:]))
    # imgProd_max_index_normal = imgProd_max_index_normal[1:]
    imgProd_max_pindex = set(imgProd_max_pindex[1:])

    # ind_arr = set(ind_arr)
    ind_arr_2 = set(list(img_arr_2))
    print(len(ind_arr_2))
    # ind_arr_3 = set(ind_arr_3)
    # ind_arr_4 = set(ind_arr_4)

    # problem is branch isnt even visited.... for some img...

    # if (imgProd_max_index != imgProd_max_index_normal or imgProd_max_index_normal != imgProd_max_index_fft or imgProd_max_index_fft != imgProd_max_pindex or imgProd_max_pindex != ind_arr 
    #     or ind_arr != ind_arr_2 or ind_arr_2 != ind_arr_3):
    # if (ind_arr != ind_arr_2 or ind_arr_2 != ind_arr_3 or ind_arr_3 != ind_arr_4):
    # if (ind_arr != ind_arr_2 or ind_arr_2 != ind_arr_3):
    if (ind_arr_2 != imgProd_max_pindex):
        ind_arr_2 = list(ind_arr_2)
        
        # print(f"ncc rocket fft: {imgProd_max_index}")
        # imgProd_max_index_normal = list(imgProd_max_index_normal)
        # print(f"ncc normal: {imgProd_max_index_normal}")
        # print(f"ncc fft: {imgProd_max_index_fft}")
        imgProd_max_pindex = list(imgProd_max_pindex)
        print(f"ncc parallel rocket fft: {imgProd_max_pindex}")
        print(f"mtree normal: {ind_arr_2}")
        # print(f"mtree normal:{ind_arr_2}")
        # print(f"mtree w fft: {ind_arr_3}")
        for j in ind_arr_2:
            for i in range(len(imgProd_max_pindex)):
                #print(metrics.distance(testSample[j], testSample[imgProd_max_index_normal[i]]))
                print(f"DISTANCE TO IMG from index {j}: {metrics.distance(testSample[j], unseen_image)}")
                print(f"DISTANCE TO IMG W INDEX from index {j}: {metrics.dist_with_index((testSample[j], j), (unseen_image, -1))}")
                print(f"DISTANCE TO IMG TEST from index {i}: {metrics.distance(testSample[imgProd_max_pindex[i]], unseen_image)}")
        # for i in range(len(img_arr)):
        #     print(metrics.distance(img_arr[i], ))

        # print(f"mtree w mst normal: {ind_arr_4}")


# print(unseen_img_parr)
# print(unseen_img_arr)
# print(unseen_img_arr_fft)
# print(unseen_img_arr_normal)


In [ ]:
k=3
runs = 100
ttime = 0
image_size = 100
sample_size = 500


data = get_data_MStar(image_size)

sample_indices = random.sample(range(len(data)), sample_size)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
testSampleIndexed = []
for i in range(len(testSample)):
    testSampleIndexed.append((testSample[i], i))

index1 = np.random.choice(sample_indices)
unseen_image = data[index1][0]
unseen_image_indexed = [unseen_image, index1]

start = time.perf_counter()
normal_tree = getMTree(testSampleIndexed, 3, d=metrics.dist_fft_numba_indexed)
end = time.perf_counter()
print(end - start)
# print(len(normal_tree))
img_arr_2 = getKNearestNeighbours(normal_tree, unseen_image_indexed, k=3)
img_arr_2 = [item[1] for item in img_arr_2]



In [ ]:
image_size = 200
sample_size = 100000

data = get_data_SARDet_100k(image_size)



In [ ]:
sample_indices = random.sample(range(len(data)), sample_size)
sampled_test_data = Subset(data, sample_indices)

testSample = [item[0] for item in sampled_test_data]
# testSampleIndexed = []
# for i in range(len(testSample)):
#     testSampleIndexed.append((testSample[i], i))
index1 = np.random.choice(sample_indices)
# index1 = np.random.randint(len(testSample))
# print(f"UNSEEN IMAGE INDEX: {index1}")
#index1 = np.random.randint(len(data))
unseen_image = data[index1][0]
# unseen_image = testSample[index1]
# unseen_image_indexed = [unseen_image, index1]

print(len(data))

In [ ]:
arr = np.ones(len(testSample))

k=7
print(len(testSample))

start = time.perf_counter()
unseen_img_arr = linear_ncc_search(testSample, unseen_image, arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
end = time.perf_counter()
print(f"fft time: {end - start}")

start = time.perf_counter()
unseen_img_parr = linear_ncc_psearch(testSample, unseen_image, arr)
imgProd_max_pindex = np.argpartition(unseen_img_parr, -(k+1))[-(k+1):]
end = time.perf_counter()
print(f"parallel time: {end-start}")

print(imgProd_max_index)
print(imgProd_max_pindex)



In [ ]:
def mtree_query_times_max_node_size(image_size=32, k=7, runs=100, max_node_sizes=[], sample_size = 1000):

    print(f"Query times of mtree w {image_size}, {k} neighbours, variable mtree max node size, sample size {sample_size} over {runs} runs")
    data = get_data_SARDet_100k(image_size)
    print(len(data))

    # avgs_mtree = []
    avgs_mtree_fft = []
    min_time = 1000
    min_node = 0

    for i in range(len(max_node_sizes)):
    
        time_mtree_fft = 0
        
        sample_indices = random.sample(range(len(data)), sample_size)
        testSample = Subset(data, sample_indices)
        testSample = [item[0] for item in testSample]
        print(len(testSample))
        print("done with testSample")
        tree_fft = getMTreeFFTNumba(testSample, max_node_sizes[i])
        

        print(f"Now testing with max nod size: {max_node_sizes[i]}")
        for j in range(runs):

            index1 = np.random.randint(len(data))
            unseen_image = data[index1][0]

            start_time = time.perf_counter()
            knn = getKNearestNeighbours(tree_fft, unseen_image, k)
            end_time = time.perf_counter()
            time_mtree_fft += end_time - start_time
    

        avg_mtree_fft = time_mtree_fft / runs
        avgs_mtree_fft.append(avg_mtree_fft)
        if (avg_mtree_fft < min_time):
            min_time = avg_mtree_fft
            min_node = max_node_sizes[i]
        print(f"Avg time for mtree fft: {avg_mtree_fft}")

    print(avgs_mtree_fft)
    return min_node

In [ ]:
max_node_sizes = list(np.arange(61))[4:]
min_node = mtree_query_times_max_node_size(image_size=32, k=7, runs=100, max_node_sizes=max_node_sizes, sample_size = 1000)
print(min_node)

In [ ]:
def plot_data_max_node_query(x_axis="", title="", filename="", varied_arr=[], data=[]):
    data_mtree_init = []
    for i in range(len(varied_arr)):
        data_mtree_init.append([varied_arr[i], data[i]])

    data_mtree_init = np.array(data_mtree_init)

    x_mtree_init, y_mtree_init = data_mtree_init.T

    plt.plot(x_mtree_init, y_mtree_init, label = f"mtree query", linestyle="-")
    plt.legend()
    plt.xlabel(x_axis)
    plt.ylabel("Runtime (s)")
    plt.title(title)
    # note its over 3 runs
    plt.tight_layout()
    plt.show()
    #plt.savefig(filename, bbox_inches='tight', dpi=1000)

In [ ]:
mtree_data = [0.06648644635453821, 0.04985567561350763, 0.04870200272882357, 0.042445177359040825, 0.0425087088951841, 0.046353489940520375, 0.040480049555189905, 0.04304797149496153, 0.04135642212582752, 0.04262374066282064, 0.044578851461410524, 0.04061049841810018, 0.04110180689021945, 0.04106593079632148, 0.03709261615527794, 0.037639004660304634, 0.03677894547581673, 0.03674548069713637, 0.04001359798945486, 0.03869960339041427, 0.03689934399910271, 0.03658173124538735, 0.036888814736157655, 0.03928412785753608, 0.03513387402286753, 0.03632704545278102, 0.03477122830459848, 0.035498228406067936, 0.03584905448602513, 0.03439247708767652, 0.03489864443661645, 0.034392852096352726, 0.038850501228589567, 0.04059506134595722, 0.037090135156176984, 0.03257076226640493, 0.035022176750935614]
varied_arr = list(np.arange(41))[4:]
plot_data_max_node_query(x_axis="max node size", title="Average runtime of finding 7 NN with sample size 100 against max node size", filename="/home/jovyan/evaluation/results/MSTAR_query_max_node_size.png", varied_arr=varied_arr, data=arr4)

In [ ]:
# mstar optimal max node size results for query on img size 32, ss 1000
# 4 - 40
arr1 =[0.07453360404120758, 0.0760427448619157, 0.057443349526729436, 0.045245827161706983, 0.04202748618321493, 0.04436554169282317, 0.0449096650374122, 0.049698567090090365, 0.04333758944878355, 0.041379778508562594, 0.03800240215845406, 0.04514048206154257, 0.039640117364469915, 0.03959023931762204, 0.03827136555686593, 0.03823503922438249, 0.03754547564079985, 0.04120285571785644, 0.03816691774642095, 0.035826291469857094, 0.036856996566057204, 0.03455932650016621, 0.037203651634044946, 0.03615273126168177, 0.03421859634341672, 0.0338427792978473, 0.03774892676388845, 0.033893928003963084, 0.03348122368799523, 0.03554718185449019, 0.03566529909381643, 0.03361380087211728, 0.03321378830587492, 0.03533396480837837, 0.03611931434832513, 0.03526111783226952, 0.03197259914362803]
# 30smt to 60
arr2= [0.0346488059964031, 0.03654287959448993, 0.03562711640959606, 0.032780495339538904, 0.034241919352207335, 0.03458906295709312, 0.03467396464431658, 0.0342395016993396, 0.033899460977409035, 0.03237264700001106, 0.03372701143147424, 0.03436243370873854, 0.03563438596436754, 0.03458592792507261, 0.03491504484321922, 0.03612813181476668, 0.03313937911298126, 0.03355619212146849, 0.03295315083116293, 0.0320811181794852, 0.032378274663351476, 0.03378636085428297, 0.0368100996594876, 0.03444660003529862, 0.03417618005303666, 0.03274708759970963, 0.03469003511127085, 0.03473432528786361, 0.03211943145142868, 0.03331151863792911, 0.03137900785310194]
# 60 to 80
arr3 = [0.03498226410476491, 0.03348756037419662, 0.03340420721564442, 0.03191877581179142, 0.032104695639573036, 0.0330363729596138, 0.0330572799523361, 0.03206634910544381, 0.03126085305120796, 0.030408310950733722, 0.03215049129677936, 0.030636014845222236, 0.030524236671626567, 0.03307789267739281, 0.032382142718415705, 0.034685473120771346, 0.03245834693312645, 0.030518119069747628, 0.03169271961785853, 0.03134439779445529, 0.03312397683737799]

arr4 = arr1 + arr2 + arr3


# imdb 4-40... for imdb optimal seems to be 39
imdb_mtree_queires = [0.06648644635453821, 0.04985567561350763, 0.04870200272882357, 0.042445177359040825, 0.0425087088951841, 0.046353489940520375, 0.040480049555189905, 0.04304797149496153, 0.04135642212582752, 0.04262374066282064, 0.044578851461410524, 0.04061049841810018, 0.04110180689021945, 0.04106593079632148, 0.03709261615527794, 0.037639004660304634, 0.03677894547581673, 0.03674548069713637, 0.04001359798945486, 0.03869960339041427, 0.03689934399910271, 0.03658173124538735, 0.036888814736157655, 0.03928412785753608, 0.03513387402286753, 0.03632704545278102, 0.03477122830459848, 0.035498228406067936, 0.03584905448602513, 0.03439247708767652, 0.03489864443661645, 0.034392852096352726, 0.038850501228589567, 0.04059506134595722, 0.037090135156176984, 0.03257076226640493, 0.035022176750935614]
# imdb 40-60
imdb_mtree_queries2 = [0.034285021647810936, 0.03713188007473946, 0.034894764921627935, 0.03555949220666662, 0.035689420572016385, 0.03569600458955392, 0.033280161919537934, 0.03747501684119925, 0.03479402721859515, 0.03337046544300392, 0.03534285521600396, 0.03567452662857249, 0.03378638550639153, 0.0334895676583983, 0.034134891235735265, 0.03503445783164352, 0.034463033261708916, 0.03489655483979732, 0.0342508251266554, 0.03370528262341395, 0.034018566329032184]


# SARDET 4-60
sardet_mtree_queries = [0.05605921014677733, 0.05075702435569838, 0.04413527720142156, 0.041756415739655495, 0.03897050929255783, 0.03522332780994475, 0.036123363303486256, 0.03425650781719014, 0.03520769790979102, 0.028383985385298727, 0.03203317269915715, 0.031361144478432836, 0.03225434086751193, 0.029638385449070482, 0.02979010680690408, 0.02958951032022014, 0.02993883696850389, 0.028521171587053685, 0.026733811788726598, 0.02914088960038498, 0.027329431518446655, 0.028482252936810254, 0.02713657370302826, 0.028098205365240575, 0.02961433698888868, 0.029213892379775643, 0.028177589017432184, 0.02646509645273909, 0.028109403648413717, 0.028149100167211144, 0.026108818335924298, 0.028049338192213325, 0.027457277516368776, 0.029412257303483784, 0.02738230670336634, 0.02729105085367337, 0.028033825629390777, 0.02971143915085122, 0.02866856729844585, 0.026201729581225665, 0.02954051753738895, 0.026153243039734663, 0.02761902290629223, 0.026829968113452195, 0.027388408198021352, 0.028706976901739834, 0.028187253254000098, 0.026537614634726196, 0.02740072306478396, 0.028295907287392764, 0.029209212146233766, 0.027169538359157742, 0.02613771740347147, 0.026517155214678498, 0.02741806108504534, 0.02730743122054264, 0.02787747632712126]



# ID Estimator Comparisons

In [ ]:
# for ID estimator comparisons:
# also functions that wouldve been nice oops
sample_sizes = [100, 1000, 2000]

def get_IMDB_data(image_size, sample_size):
    IMDB_WIKI_data = get_data(image_size)
    sample_indices = random.sample(range(len(IMDB_WIKI_data)), sample_size)
    sampled_test_data = Subset(IMDB_WIKI_data, sample_indices)
    testSample = np.array(sampled_test_data)
    return testSample

def get_MSTAR_data(image_size, sample_size):
    data = get_data_MStar(image_size)
    sample_indices = random.sample(range(len(data)), sample_size)
    sampled_test_data = Subset(data, sample_indices)
    testSample = np.array([item[0] for item in sampled_test_data])
    return testSample

def get_SARDET_data(image_size, sample_size):
    data = get_data_SARDet_100k(image_size)
    sample_indices = random.sample(range(len(data)), sample_size)
    sampled_test_data = Subset(data, sample_indices)
    testSample = np.array([item[0] for item in sampled_test_data])
    return testSample

def get_ATRNET_data(sample_size, list_path):
    data = np.load(list_path)
    all_data = data["testSample"]
    sample_indices = np.array(random.sample(range(len(all_data)), sample_size))
    testSample = all_data[sample_indices]
    return testSample


In [ ]:
def mtree_ncc_query_times_sample_size(image_size=128, k=7, runs=100, max_node_size=12, list_data="path", sample_sizes = []):

    print(f"Query times of diff methods based on kNN with ncc with image size {image_size}, {k} neighbours, mtree max node size {max_node_size} over {runs} runs")

    data_list = np.load(list_data)
    data = data_list["testSample"]

            
    avgs_ncc_pfft = []
    avgs_ncc_fft = []
    avgs_ncc_unoptim = []
    avgs_mtree = []
    avgs_mtree_fft = []

    for i in range(len(sample_sizes)):
        
        time_ncc_pfft = 0
        time_ncc_fft = 0
        time_ncc_unoptim = 0
        time_mtree = 0
        time_mtree_fft = 0

        sample_indices = random.sample(range(len(data)), sample_sizes[i])
        testSample = data[sample_indices]
        print("done with testSample")
        # tree = getMTree(testSample, max_node_size)
        tree_fft = getMTreeFFTNumba(testSample, max_node_size)
        

        print(f"Now testing with sample size: {sample_sizes[i]}")
        for j in range(runs):

            index1 = np.random.randint(len(data))
            unseen_image = data[index1]

            start_time = time.perf_counter()
            arr = np.ones(len(testSample))
            unseen_img_arr = ImageProducts.linear_ncc_psearch(testSample, unseen_image, arr)
            imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
            end_time = time.perf_counter()
            time_ncc_pfft += end_time - start_time
            
            start_time = time.perf_counter()
            arr = np.ones(len(testSample))
            unseen_img_arr = ImageProducts.linear_ncc_search(testSample, unseen_image, arr)
            imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
            end_time = time.perf_counter()
            time_ncc_fft += end_time - start_time

            # start_time = time.perf_counter()
            # arr = []
            # for j in range(len(testSample)):
            #     result = ImageProducts.ncc_scaled(testSample[j], unseen_image)
            #     arr.append(result)
            
            # unseen_img_arr = np.array(arr)
            # imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
            # time_ncc_unoptim += end_time - start_time

            # start_time = time.perf_counter()
            # knn = getKNearestNeighbours(tree, unseen_image, k)
            # end_time = time.perf_counter()
            # time_mtree += end_time - start_time

            start_time = time.perf_counter()
            knn = getKNearestNeighbours(tree_fft, unseen_image, k)
            end_time = time.perf_counter()
            time_mtree_fft += end_time - start_time
        
        avg_ncc_pfft = time_ncc_pfft / runs
        avgs_ncc_pfft.append(avg_ncc_pfft)
        print(f"Avg time for ncc pfft: {avg_ncc_pfft}")

        avg_ncc_fft = time_ncc_fft / runs
        avgs_ncc_fft.append(avg_ncc_fft)
        print(f"Avg time for ncc fft: {avg_ncc_fft}")

        # avg_ncc_unoptim = time_ncc_unoptim / runs
        # avgs_ncc_unoptim.append(avg_ncc_unoptim)
        # print(f"Avg time for ncc unoptim: {avg_ncc_unoptim}")

        # avg_mtree = time_mtree / runs
        # avgs_mtree.append(avg_mtree)
        # print(f"Avg time for mtree: {avg_mtree}")

        avg_mtree_fft = time_mtree_fft / runs
        avgs_mtree_fft.append(avg_mtree_fft)
        print(f"Avg time for mtree fft: {avg_mtree_fft}")

    print(avgs_ncc_pfft)
    print(avgs_ncc_fft)
    # print(avgs_ncc_unoptim)
    # print(avgs_mtree)
    print(avgs_mtree_fft)
    with open("/home/jovyan/evaluation/results/test_query_sample_sizes.txt", "w") as file:
        file.write(f"sample_sizes = {sample_sizes}\n")
        file.write(f"avgs_ncc_pfft = {avgs_ncc_pfft}\n")
        file.write(f"avgs_ncc_fft = {avgs_ncc_fft}\n")
        # file.write(f"avgs_ncc_unoptim = {avgs_ncc_unoptim}\n")
        # file.write(f"avgs_mtree = {avgs_mtree}\n")
        file.write(f"avgs_mtree_fft = {avgs_mtree_fft}")



def ncc_pfft_query_time(image_size=128, k=7, runs=100, sample_size=1000, testSample=[], unseen_image=[]):
    avg_ncc_pfft = 0
    time_ncc_pfft = 0
    for j in range(runs):
        start_time = time.perf_counter()
        arr = np.ones(len(testSample))
        unseen_img_arr = ImageProducts.linear_ncc_psearch(testSample, unseen_image, arr)
        imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
        end_time = time.perf_counter()
        time_ncc_pfft += end_time - start_time
    
    avg_ncc_pfft = time_ncc_pfft / runs
    return avg_ncc_pfft

def ncc_fft_query_time(image_size=128, k=7, runs=100, sample_size=1000, testSample=[], unseen_image=[]):
    time_ncc_fft = 0
    for j in range(runs):
        start_time = time.perf_counter()
        arr = np.ones(len(testSample))
        unseen_img_arr = ImageProducts.linear_ncc_search(testSample, unseen_image, arr)
        imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
        end_time = time.perf_counter()
        time_ncc_fft += end_time - start_time
    
    avg_ncc_fft = time_ncc_fft / runs
    return avg_ncc_fft

def mtree_fft_query_time(image_size=128, k=7, runs=100, max_node_size=25, sample_size=1000, testSample=[], unseen_image=[]):
    time_mtree_fft = 0
    tree_fft = getMTreeFFTNumba(testSample, max_node_size)
    for j in range(runs):
        start_time = time.perf_counter()
        knn = getKNearestNeighbours(tree_fft, unseen_image, k)
        end_time = time.perf_counter()
        time_mtree_fft += end_time - start_time
    
    avg_mtree_fft = time_mtree_fft / runs
    return avg_mtree_fft

In [ ]:
def query_percentage_improvements_mtree_ncc(sample_sizes=[], runs=10, k=7):
    print(f"Measuring percentage improvement of mtree over ncc for IMDB, MSTAR, SARDET, ATRNET over {runs} runs for {k} neighbours over variable sample sizes and image size 128")
    list_path = "/home/jovyan/data/ATRNet-STAR_annotations/list_data_all.npz"
    p_incs_imdb = []
    p_incs_MSTAR = []
    p_incs_SARDET = []
    p_incs_ATRNET = []
    IMDB_WIKI_data = get_data(128)
    MSTAR_data = get_data_MStar(128)
    SARDET_data = get_data_SARDet_100k(128)
    ATRNET_data = np.load(list_path)
    all_data = ATRNET_data["testSample"]

    for i in range(len(sample_sizes)):
        print(f"Now doing sample size {sample_sizes[i]}")
        
        sample_indices = random.sample(range(len(IMDB_WIKI_data)), sample_sizes[i])
        sampled_test_data = Subset(IMDB_WIKI_data, sample_indices)
        testSample_imdb = np.array(sampled_test_data)
        unseen_image_imdb = IMDB_WIKI_data[np.random.randint(len(IMDB_WIKI_data))]

        
        sample_indices = random.sample(range(len(MSTAR_data)), sample_sizes[i])
        sampled_test_data = Subset(MSTAR_data, sample_indices)
        testSample_MSTAR = np.array([item[0] for item in sampled_test_data])
        unseen_image_MSTAR = MSTAR_data[np.random.randint(len(MSTAR_data))][0]

        
        sample_indices = random.sample(range(len(SARDET_data)), sample_sizes[i])
        sampled_test_data = Subset(SARDET_data, sample_indices)
        testSample_SARDET = np.array([item[0] for item in sampled_test_data])
        unseen_image_SARDET = SARDET_data[np.random.randint(len(SARDET_data))][0]

        sample_indices = np.array(random.sample(range(len(all_data)), sample_sizes[i]))
        testSample_ATRNET = all_data[sample_indices]
        unseen_image_ATRNET = all_data[np.random.randint(len(all_data))]
    

        avg_ncc_fft_time_imdb = ncc_fft_query_time(image_size=128, k=k, runs=runs, sample_size=sample_sizes[i], testSample=testSample_imdb, unseen_image=unseen_image_imdb)
        avg_ncc_fft_time_MSTAR = ncc_fft_query_time(image_size=128, k=k, runs=runs, sample_size=sample_sizes[i], testSample=testSample_MSTAR, unseen_image=unseen_image_MSTAR)
        avg_ncc_fft_time_SARDET = ncc_fft_query_time(image_size=128, k=k, runs=runs, sample_size=sample_sizes[i], testSample=testSample_SARDET, unseen_image=unseen_image_SARDET)
        avg_ncc_fft_time_ATRNET = ncc_fft_query_time(image_size=128, k=k, runs=runs, sample_size=sample_sizes[i], testSample=testSample_ATRNET, unseen_image=unseen_image_ATRNET)
        print(f"TIMES NCC: {avg_ncc_fft_time_imdb}, {avg_ncc_fft_time_MSTAR}, {avg_ncc_fft_time_SARDET}, {avg_ncc_fft_time_ATRNET}")

        avg_mtree_fft_time_imdb = mtree_fft_query_time(image_size=128, k=k, runs=runs, max_node_size=39, sample_size=sample_sizes[i], testSample=testSample_imdb, unseen_image=unseen_image_imdb)
        avg_mtree_fft_time_MSTAR = mtree_fft_query_time(image_size=128, k=k, runs=runs, max_node_size=39, sample_size=sample_sizes[i], testSample=testSample_MSTAR, unseen_image=unseen_image_MSTAR)
        avg_mtree_fft_time_SARDET = mtree_fft_query_time(image_size=128, k=k, runs=runs, max_node_size=39, sample_size=sample_sizes[i], testSample=testSample_SARDET, unseen_image=unseen_image_SARDET)
        avg_mtree_fft_time_ATRNET = mtree_fft_query_time(image_size=128, k=k, runs=runs, max_node_size=25, sample_size=sample_sizes[i], testSample=testSample_ATRNET, unseen_image=unseen_image_ATRNET)
        print(f"TIMES MTREE: {avg_mtree_fft_time_imdb}, {avg_mtree_fft_time_MSTAR}, {avg_mtree_fft_time_SARDET}, {avg_mtree_fft_time_ATRNET}")

        p_inc_imdb = ((avg_ncc_fft_time_imdb - avg_mtree_fft_time_imdb) / avg_mtree_fft_time_imdb) * 100
        p_incs_imdb.append(p_inc_imdb)
        print(f"increase for imdb: {p_inc_imdb}")

        p_inc_MSTAR = ((avg_ncc_fft_time_MSTAR - avg_mtree_fft_time_MSTAR) / avg_mtree_fft_time_MSTAR) * 100
        p_incs_MSTAR.append(p_inc_MSTAR)
        print(f"increase for MSTAR: {p_inc_MSTAR}")

        p_inc_SARDET = ((avg_ncc_fft_time_SARDET - avg_mtree_fft_time_SARDET) / avg_mtree_fft_time_SARDET) * 100
        p_incs_SARDET.append(p_inc_SARDET)
        print(f"increase for SARDET: {p_inc_SARDET}")

        p_inc_ATRNET = ((avg_ncc_fft_time_ATRNET - avg_mtree_fft_time_ATRNET) / avg_mtree_fft_time_ATRNET) * 100
        p_incs_ATRNET.append(p_inc_ATRNET)
        print(f"increase for ATRNET: {p_inc_ATRNET}")
    
    print(f"{p_incs_imdb}")
    print(f"{p_incs_MSTAR}")
    print(f"{p_incs_SARDET}")
    print(f"{p_incs_ATRNET}")

    with open("/home/jovyan/evaluation/results/ID_estimator_comparisons.txt", "w") as file:
        file.write(f"sample_sizes = {sample_sizes}\n")
        file.write(f"p_incs_imdb = {p_incs_imdb}\n")
        file.write(f"p_incs_MSTAR = {p_incs_MSTAR}\n")
        file.write(f"p_incs_SARDET = {p_incs_SARDET}\n")
        file.write(f"p_incs_ATRNET = {p_incs_ATRNET}")


In [ ]:
sample_sizes = [100, 1000, 2000]
query_percentage_improvements_mtree_ncc(sample_sizes=sample_sizes, runs=100, k=7)

In [ ]:
def mtree_ncc_query_times_sample_size(image_size=128, k=7, runs=100, max_node_size=12, list_data="path", sample_sizes = [], filename=""):

    print(f"Query times of diff methods based on kNN with ncc with image size {image_size}, {k} neighbours, mtree max node size {max_node_size} over {runs} runs")

    # data_list = np.load(list_data)
    # data = data_list["testSample"]
    data = get_data_MStar(image_size)

            
    avgs_ncc_pfft = []
    avgs_ncc_fft = []
    avgs_ncc_unoptim = []
    avgs_mtree = []
    avgs_mtree_fft = []

    for i in range(len(sample_sizes)):
        
        time_ncc_pfft = 0
        time_ncc_fft = 0
        time_ncc_unoptim = 0
        time_mtree = 0
        time_mtree_fft = 0

        sample_indices = random.sample(range(len(data)), sample_sizes[i])
        sampled_test_data = Subset(data, sample_indices)
        testSample = np.array([item[0] for item in sampled_test_data])
        print("done with testSample")
        # tree = getMTree(testSample, max_node_size)
        tree_fft = getMTreeFFTNumba(testSample, max_node_size)
        

        print(f"Now testing with sample size: {sample_sizes[i]}")
        for j in range(runs):

            index1 = np.random.randint(len(data))
            unseen_image = data[index1][0]

            start_time = time.perf_counter()
            arr = np.ones(len(testSample))
            unseen_img_arr = ImageProducts.linear_ncc_psearch(testSample, unseen_image, arr)
            imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
            end_time = time.perf_counter()
            time_ncc_pfft += end_time - start_time
            
            start_time = time.perf_counter()
            arr = np.ones(len(testSample))
            unseen_img_arr = ImageProducts.linear_ncc_search(testSample, unseen_image, arr)
            imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
            end_time = time.perf_counter()
            time_ncc_fft += end_time - start_time

            # start_time = time.perf_counter()
            # arr = []
            # for j in range(len(testSample)):
            #     result = ImageProducts.ncc_scaled(testSample[j], unseen_image)
            #     arr.append(result)
            
            # unseen_img_arr = np.array(arr)
            # imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):]
            # time_ncc_unoptim += end_time - start_time

            # start_time = time.perf_counter()
            # knn = getKNearestNeighbours(tree, unseen_image, k)
            # end_time = time.perf_counter()
            # time_mtree += end_time - start_time

            start_time = time.perf_counter()
            knn = getKNearestNeighbours(tree_fft, unseen_image, k)
            end_time = time.perf_counter()
            time_mtree_fft += end_time - start_time
        
        avg_ncc_pfft = time_ncc_pfft / runs
        avgs_ncc_pfft.append(avg_ncc_pfft)
        print(f"Avg time for ncc pfft: {avg_ncc_pfft}")

        avg_ncc_fft = time_ncc_fft / runs
        avgs_ncc_fft.append(avg_ncc_fft)
        print(f"Avg time for ncc fft: {avg_ncc_fft}")

        # avg_ncc_unoptim = time_ncc_unoptim / runs
        # avgs_ncc_unoptim.append(avg_ncc_unoptim)
        # print(f"Avg time for ncc unoptim: {avg_ncc_unoptim}")

        # avg_mtree = time_mtree / runs
        # avgs_mtree.append(avg_mtree)
        # print(f"Avg time for mtree: {avg_mtree}")

        avg_mtree_fft = time_mtree_fft / runs
        avgs_mtree_fft.append(avg_mtree_fft)
        print(f"Avg time for mtree fft: {avg_mtree_fft}")

    print(avgs_ncc_pfft)
    print(avgs_ncc_fft)
    # print(avgs_ncc_unoptim)
    # print(avgs_mtree)
    print(avgs_mtree_fft)
    with open(filename, "w") as file:
        file.write(f"sample_sizes = {sample_sizes}\n")
        file.write(f"avgs_ncc_pfft = {avgs_ncc_pfft}\n")
        file.write(f"avgs_ncc_fft = {avgs_ncc_fft}\n")
        # file.write(f"avgs_ncc_unoptim = {avgs_ncc_unoptim}\n")
        # file.write(f"avgs_mtree = {avgs_mtree}\n")
        file.write(f"avgs_mtree_fft = {avgs_mtree_fft}")

In [ ]:
# now testing for mstar
mtree_ncc_query_times_sample_size(image_size=128, k=7, runs=30, max_node_size=15, list_data="path", sample_sizes = [6000, 7000, 8000, 9000], filename="/home/jovyan/evaluation/results/MSTAR_test_query_sample_sizes_3.txt")

In [ ]:
data = get_data(128)

sample_indices = random.sample(range(len(data)), 1000)
sampled_test_data = Subset(data, sample_indices)
testSample = np.array(sampled_test_data)

index1 = np.random.randint(len(data))
unseen_image = data[index1]

In [ ]:
import psutil

k = 7

start_time = time.perf_counter()
arr = np.ones(len(testSample))
unseen_img_arr = ImageProducts.linear_ncc_psearch(testSample, unseen_image, arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
end_time = time.perf_counter()
print(end_time - start_time)

start_time = time.perf_counter()
arr = np.ones(len(testSample))
unseen_img_arr = ImageProducts.linear_ncc_search(testSample, unseen_image, arr)
imgProd_max_index = np.argpartition(unseen_img_arr, -(k+1))[-(k+1):][1:]
end_time = time.perf_counter()
print(f"for non parallel: {end_time - start_time}")
# my_function()

In [ ]:
import numba
print(numba.config.NUMBA_NUM_THREADS)

In [10]:
def save_list_data(image_size):

    data = get_data(image_size)


    testSample = [item for item in data]
    print("Done w testSample")


    filename = "/home/jovyan/data/annotations/imdb_list_data_all_128.npz"
    np.savez(filename, testSample=testSample)

In [ ]:
save_list_data(128)

Done w testSample
